<a href="https://colab.research.google.com/github/LeeMinQi-25/Fake-News-Detection-with-Deep-Learning/blob/main/NLP_Group_Project_Group16(BERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake News Detection

In [ ]:
!pip install gradio


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import gradio as gr

# ----- DATA LOADING & PREPROCESSING -----
df = pd.read_csv("WELFake_Dataset.csv")  # Replace with your actual CSV filename
df = df.dropna(subset=['title', 'text', 'label']).reset_index(drop=True)
df['Text'] = df['title'] + " " + df['text']
df = df[['Text', 'label']].rename(columns={'label': 'Label'})
df['Label'] = df['Label'].astype(int)

print("Dataset shape after cleaning:", df.shape)

# ----- TRAIN-TEST SPLIT -----
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Text'].tolist(), df['Label'].tolist(), test_size=0.2
)

Dataset shape after cleaning: (71537, 2)


BERT

In [ ]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification, create_optimizer
import tensorflow as tf

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=256)

# Prepare datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(8)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(8)

# Load model
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Use Hugging Face's optimizer creation utility
num_train_steps = len(train_dataset) * 3  # 3 epochs
optimizer, schedule = create_optimizer(init_lr=2e-5, num_train_steps=num_train_steps, num_warmup_steps=0)

# Compile model
bert_model.compile(optimizer=optimizer,
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=['accuracy'])

# Train model
bert_history = bert_model.fit(train_dataset, epochs=3, validation_data=test_dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
7154/7154 [==============================] - 3589s 495ms/step - loss: 0.0462 - accuracy: 0.9818 - val_loss: 0.0174 - val_accuracy: 0.9938
Epoch 2/3
7154/7154 [==============================] - 3540s 495ms/step - loss: 0.0112 - accuracy: 0.9960 - val_loss: 0.0162 - val_accuracy: 0.9954
Epoch 3/3
7154/7154 [==============================] - 3541s 495ms/step - loss: 0.0023 - accuracy: 0.9992 - val_loss: 0.0186 - val_accuracy: 0.9949


In [ ]:
# Save the model
bert_model.save_pretrained("saved_bert_model")

# Save the tokenizer
tokenizer.save_pretrained("saved_bert_model")


('saved_bert_model/tokenizer_config.json',
 'saved_bert_model/special_tokens_map.json',
 'saved_bert_model/vocab.txt',
 'saved_bert_model/added_tokens.json')

In [ ]:
import shutil

# Zip the folder
shutil.make_archive("saved_bert_model", 'zip', "saved_bert_model")


'/content/saved_bert_model.zip'